In [1]:
# Fundamental
import sys, re, random, operator, time
import datetime, json, pprint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#ML
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cross_validation import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif

# Query
import urllib3 as url
import xml.etree.ElementTree as ET

# Visualize
import seaborn as sns

### Parameters + Instancing

In [2]:
http = url.PoolManager()
pp = pprint.PrettyPrinter(indent=4)
pd.options.display.max_seq_items = 100
%matplotlib inline

In [24]:
# Propublica paths
ppdelay = .25 # API delay
pplimit = 74 # Pause Requests
pppath = "https://projects.propublica.org/nonprofits/api/v2"
ppsearch = lambda x: "/search.json?q="+x
pporg = lambda x: "/organizations/"+str(x)+".json"
ppsearch = lambda x: "/search.json?q="+str(x).replace(" ", "%20")

# S3 Paths
s3path = "https://s3.amazonaws.com/irs-form-990"
s3index = lambda x: "/index_"+str(x)+".csv"
s3xml = lambda x:"/"+str(x)+"_public.xml"

### Core Functions
##### Methods for handling Propublica API

In [4]:
def get_eins_by_search(q,state=None,ntee=None,ccode=None):
    # q is string query
    # State string ed, eg NY
    # ntee is National Taxonomy of Exempt Entities (NTEE) Major Group, by Integer
    # ccode is organization category code by Integer, eg. 3 is 501(c)3
    # https://projects.propublica.org/nonprofits/api#methods
    query = ""
    query+="&state%5Bid%5D="+str(state) if state!=None else ""
    query+="ntee%5Bid%5D="+str(ntee) if ntee!=None else ""
    query+="c_code%5Bid%5D="+str(ccode) if ccode!=None else ""
    print(pppath+ppsearch(q)+query)
    response = http.request('GET',pppath+ppsearch(q)+query)
    parsed = json.loads(response.data.decode('utf-8'))
    r = json.dumps(parsed["organizations"], indent=4, sort_keys=True)
#     print(r)
    return pd.read_json(r,orient="index").T

In [5]:
def get_by_ein(ein,year=None):
    # Returns dataframe of FILING OBJECTS for that Organziation
    # https://projects.propublica.org/nonprofits/api#filing-object
    response = http.request('GET',pppath+pporg(ein))
    parsed = json.loads(response.data.decode('utf-8'))
    r = json.dumps(parsed["filings_with_data"], indent=4, sort_keys=True)
    # If there is nothing in the response
#     print(r,len(r))
    if(len(r)<=2): 
        return pd.DataFrame([("ein",ein),("tax_prd_yr",year)])
    else:
        df =  pd.read_json(r,orient="index").T.set_index('tax_prd_yr')
        if year != None: # If a year is specified
            if (int(year) in df.index) == False: # Make sure that year is there, if not return empty row
                return pd.DataFrame([("ein",ein),("tax_prd_yr",year)])
            else: # otherwise return year as series
                return df.loc[year] # Return year
        else: # Otherwise return all years available
            return df
#     df.loc[year] if year!=None else df
#     return df.loc[year] if year!=None else df

In [22]:
def get_by_eins_and_year(einlist,year):
    # Accepts list of EINs
    # Populates filing objects per row
    df = pd.DataFrame()
    count = 0
    for ein in einlist:
        print(count, ein,year)
        count+=1
        try:
            row = get_by_ein(ein,year)
        except:
            print("Rate limit at " + str(count) + " with " + str(ppdelay) + " seconds")
        print(row)
        df = df.append(row)
        if count%pplimit != 0: time.sleep(ppdelay)
        else: time.sleep(10) # Cooldown
    return df

In [7]:
def search_by_name(name, zipcode, returnlist=False):
    # Retrieve EIN results by name
    # If returnlist==True, Return full list or rtruncated list
    # If returnlist==False, return most likely
    return ein

##### Methods for handling AWS hosted IRS xml

In [8]:
def get_unique_from_ein(year,ein):
    df = pd.read_csv(s3path+s3index(str(year)))
    objid = df[df["EIN"]==ein].OBJECT_ID
    return objid.values[0]

In [9]:
def get_990_to_df(objid):
    response = http.request('GET',s3path+s3xml(objid))
    root = ET.fromstring(response.data.decode('utf-8')) # element tree
    print(root)
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            record[subchild.tag] = subchild.text
            all_records.append(record)
    return pd.DataFrame(all_records)

In [10]:
def get_990_to_xml(objid):
    response = http.request('GET',s3path+s3xml(objid))
    root = ET.fromstring(response.data.decode('utf-8')) # element tree
    return root

### Explore
[Dictionary](https://github.com/CharityNavigator/irs990/blob/master/docs/explore-database.md)

In [ ]:
r = get_unique_from_ein(2015,382912028)
df=get_990_to_df(r)
df

In [ ]:
t=get_by_ein(382912028)


In [ ]:
t

In [ ]:
# Example of Searching and then plotting
t = get_by_ein(get_eins_by_search("New America",state="DC")['ein'][0])

In [ ]:
sns.stripplot(x="tax_prd_yr",y="totfuncexpns", data=t.reset_index());

In [25]:
newdf = get_by_eins_and_year(get_eins_by_search("New America",state="DC")["ein"], 2015)

https://projects.propublica.org/nonprofits/api/v2/search.json?q=New%20America&state%5Bid%5D=DC
0 522096845 2015
compnsatncurrofcr                       1523060
cstbasisecur                              16041
cstbasisothr                              54640
ein                                   522096845
formtype                                      0
gftgrntsrcvd170                       111432953
gnlsecur                                   -631
gnlsothr                                 -54640
grsalesecur                               15410
grsalesinvent                                 0
grsalesothr                                   0
grsinc170                               1459643
grsincfndrsng                                 0
grsincgaming                                  0
grsincmembers                                 0
grsincother                                   0
grsrcptsadmissn509                            0
grsrcptspublicuse                             0
grsrcptsrelated170      

/home/ubuntu/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/ubuntu/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1 731629523 2015
            0          1
0         ein  731629523
1  tax_prd_yr       2015


/home/ubuntu/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2 455419038 2015
basisalesexpnsothr                                                       0
costgoodsold                                                             0
direxpns                                                                 0
duesassesmnts                                                            0
ein                                                              455419038
formtype                                                                 1
gftgrntrcvd170                                                         NaN
gnsaleofastothr                                                          0
grsalesminusret                                                          0
grsamtsalesastothr                                                       0
grsinc170                                                                0
grsincgaming                                                             0
grsprft                                                                  0
grspubli

...

/home/ubuntu/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


98 300145807 2015
Rate limit at 99 with 0.25 seconds
compnsatncurrofcr                        253281
cstbasisecur                                  0
cstbasisothr                                  0
ein                                   363304707
formtype                                      0
gftgrntsrcvd170                               0
gnlsecur                                      0
gnlsothr                                      0
grsalesecur                                   0
grsalesinvent                                 0
grsalesothr                                   0
grsinc170                                     0
grsincfndrsng                                 0
grsincgaming                                  0
grsincmembers                                 0
grsincother                                   0
grsrcptsadmissn509                      2310565
grsrcptspublicuse                             0
grsrcptsrelated170                            0
grsrntsprsnl                       

/home/ubuntu/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


99 131669037 2015
Rate limit at 100 with 0.25 seconds
compnsatncurrofcr                        253281
cstbasisecur                                  0
cstbasisothr                                  0
ein                                   363304707
formtype                                      0
gftgrntsrcvd170                               0
gnlsecur                                      0
gnlsothr                                      0
grsalesecur                                   0
grsalesinvent                                 0
grsalesothr                                   0
grsinc170                                     0
grsincfndrsng                                 0
grsincgaming                                  0
grsincmembers                                 0
grsincother                                   0
grsrcptsadmissn509                      2310565
grsrcptspublicuse                             0
grsrcptsrelated170                            0
grsrntsprsnl                      

/home/ubuntu/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [15]:
get_by_ein(530196583,2015)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


compnsatncurrofcr                       2920469
cstbasisecur                          102917033
cstbasisothr                                  0
ein                                   530196583
formtype                                      0
gftgrntsrcvd170                        91413187
gnlsecur                               23380224
gnlsothr                                  70739
grsalesecur                           126297257
grsalesinvent                            994092
grsalesothr                               70739
grsinc170                              17341584
grsincfndrsng                            102679
grsincgaming                                  0
grsincmembers                                 0
grsincother                                   0
grsrcptsadmissn509                            0
grsrcptspublicuse                             0
grsrcptsrelated170                   1094213945
grsrntsprsnl                                  0
grsrntsreal                             

In [ ]:
df.to_frame()

In [26]:
newdf

,compnsatncurrofcr,cstbasisecur,cstbasisothr,ein,formtype,gftgrntsrcvd170,gnlsecur,gnlsothr,grsalesecur,grsalesinvent,...,othrchgsnetassetfnd,othrevnue,othrinvstinc,prgmservrev,totcntrbs,totexcessyr,totexpns,totnetassetsend,totrevnue,unrelbusincd
2015,1523060.0,16041.0,54640.0,522096845.0,0.0,111432953.0,-631.0,-54640.0,15410.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,NaN,NaN,NaN,455419038.0,1.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,91983.0,4686.0,87297.0,5555.0,91983.0,0.0
2015,346529.0,0.0,0.0,271661221.0,0.0,60040461.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,296686.0,349971.0,1503.0,521660746.0,0.0,13550187.0,817.0,-1503.0,350788.0,2242.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,0.0,0.0,0.0,232188166.0,0.0,1454210.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,718544.0,0.0,944.0,264568349.0,0.0,0.0,0.0,-944.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
sns.kdeplot(["totfuncexpns"], data=newdf);

TypeError: kdeplot() got multiple values for argument 'data'

In [ ]:
sns.jointplot("totfuncexpns", "total", data=tips, kind="reg",
                  xlim=(0, 60), ylim=(0, 12), color="r", size=7)